# Towards practical arithmetic coding schemes
In the previous notebook we've introduced Elias coding and commented on its potential to achieve the Shannon's entropy. However, we've also realised that Elias coding is not practical as it would require large (or even infinite) precision arithmetic.

In this notebook we'll see how the initial Elias coding scheme can be re-arranged to implement a practical arithmetic coding engine which will avoid:
 * Use of infinite arithmetic coding precision.
 * Use of multiplications.

## Resurrecting Elias coding
Because of its high numerical precision requirement, Elias coding remained an academic curiosity until 1976 when [Pasco](https://www.richpasco.org/scaffdc.pdf) and Rinassen proposed some finite precision implementations. Both based their implementations from a key observation associated with the second property of Elias coding (reported here for your convenience):

For each $n_1$, $n_2$ such that $n_2 > n_1$, with $s_{0:n_1}$ being a prefix for $s_{0:n_2}$, the intervals associated with $s_{0:n_1}$ and $s_{0:n_2}$ satisfies the following:

$$
\large
[low_{n_2}, high_{n_2}) \subseteq [low_{n_1}, high_{n_1})
$$

We also recall from the previous notebook that $high_k$ is decreasing whilst $low_k$ is increasing and $low_k < high_k$.

Given that each interval associated with any suffix of the message's string is contained in the prefix's interval, we can think to dump onto a file/buffer the precision accumulated so far and then carry on with a **rescaled precision interval**. The figure below illustrates this concept.

<img src="interval-rescaling.png" width="600">

With the modus operandi illustrated above we can still use Elias coding as coding scheme with all its good properties but we don't need to worry about hitting the numerical precision's of the hardware where the encoding is running. With this main idea in mind we need now to implement it. And for this to happen we'll make a refactoring of our implementation of Elias coding (from the previous notebook) which will simplify the following explanation. The refactoring will touch the following aspects:
 * Introduction of a Python dictionary to model the pmf
 * Removal of the if/else to tackle the update of `high_k` and `low_k`
 
The Python cell below implements this refactoring. Please note the different values for `high_k` and `low_k` due to the different (and probably more intuitive) convention adopted to represent the pmf: number of bits and length of the interval do not change (as expected).

In [ ]:
# Refactoring of the Python code implementing Elias coding
import numpy as np
pmf = {'0': [0, 0.25], '1': [0.25, 1]}
message = '0110'
low_k, high_k = 0, 1

for m in message:
    range_k = high_k - low_k
    high_k = low_k + range_k * pmf[m][1]
    low_k = low_k + range_k * pmf[m][0]

# Final calculations and print out
total_bits = np.ceil(-np.log2(high_k - low_k))
codeword = (high_k + low_k) / 2
bps = (total_bits + 1) / len(message)
print(f"Total bits: {total_bits:.2f}, bps: {bps:.2f}, final codeword: {codeword}")

## Getting real: Working with integer arithmetic
The refactoring has just simplified the code but didn't remove the need for large numerical precision. To do so we'll declare our `high_k` and `low_k` as:

`uint32_t high_k = 0xFFFFFFFF, low_k = 0`

The value assigned to `high_k` leads to a (binary) periodic fraction which, if we have an infinite number of ones, converges to the real value 1. For a prove of this have a look at the addendum cell below.

## Addendum:  Periodic decimal and binary fractions
In the old good days at college our teacher proved that $0.\bar{9} \rightarrow 1$. This was done by writing $0.999\ldots$ in the so-called *scientific notation*:

$$
0.999\ldots = 9\cdot10^{-1} + 9\cdot10^{-2} + 9\cdot10^{-3} + \ldots = 9\cdot\sum_{n=1}^{\infty} \left(\frac{1}{10}\right)^n.
$$

The summation above resembles to what in mathematics is called *geometric series*:

$$
\sum_{n=0}^{\infty}q^n \xrightarrow[]{|q| < 1} = \frac{1}{1 - q},
$$

where $q$ denotes the *common ratio* of the series. Our summation for the periodic fraction starts from $n=1$, so we need to subtract the result of the geometric series when $n=0$. Therefore we'll have:

$$
9\cdot\sum_{n=1}^{\infty} \left(\frac{1}{10}\right)^n = 9\cdot\sum_{n=0}^{\infty} \left(\frac{1}{10}\right)^n - 9 = 10 - 9 = 1
$$

Using the same approach we can now prove that:

$$
0.\bar{1} = \sum_{n=1}^{\infty} \left(\frac{1}{2}\right)^n = \sum_{n=0}^{\infty} \left(\frac{1}{2}\right)^n - 1 = 2 - 1 = 1
$$

With the adoption of integer arithmetic, our arithmetic encoder can be reimplemented as shown in the following Python cell below.

In [ ]:
pmf = {'0': [0, 25], '1': [25, 100]}
precision = np.uint32(100) # See comment alpha
message = '0110'
low_k, high_k = np.uint32(0), np.uint32(0xFFFFFFFF)
output_buffer = ''

for m in message:
    range_k = high_k - low_k + 1
    high_k = low_k + (range_k * pmf[m][1] // precision)
    low_k = low_k + (range_k * pmf[m][0] // precision)
    while True:
        if high_k < 0x80000000: # See comment beta
            output_buffer += '0'
        elif low_k >= 0x80000000: # See comment gamma
            output_buffer += '1'
        else:
            break
        # See comment delta
        low_k <<= 1
        high_k <<= 1
        high_k |= 1
        
# Write out the closing bit, see comment epsilon
if low_k < 0x40000000:
    output_buffer += '0'
else:
    output_buffer += '1'

print(f"Output buffer: {output_buffer}")

The code above contains some comments which are now elaborated below.

 * **Alpha**: The probability mass function is assumed to have fixed point precision, with a scale of 100 in this case.
 * **Beta**: We've already mentioned above that `high_k` is decreasing whilst `low_k` is increasing and `low_k` < `high_k`. Accordingly, if `high_k` is less than `0x80000000` it means that its MSB will be zero and (more importantly) it will be zero from now on. So we can append/write a zero bit to the output buffer and carry on with the coding process. This operation corresponds to the intuition expressed above whereby we can drop the bits accumulated in the registry (`high_k` in this case).

 * **Gamma**: An analogous consideration can be made for `low_k`. In fact if its value is greater than `0x7FFFFFFF`, then it means that its MSB will be one and will stay stuck to that value from now on. So we can append/write a `1` bit to the output buffer and carry on with the coding process.
 * **Delta**: After we've dumped the value for the MSB of `low_k` and/or `high_k`, we can remove it by the shift operations included in the Python code above. In particular `low_k` will get a zero inserted whilst `high_k` will also get added a one, since `high_k` > `low_k`.
 * **Epsilon**: This conclusive writing is a sort of end of file marker that we add to signal where the final pair `low_k` and `high_k` ended up.

The code we commented would be a working example but it doesn't address an important drawback. In fact, it could happen that both `low_k` and `high_k` approach 0.5 (i.e. `0x80000000`) but do not cross that value. More precisely, we might end up in a situation where `low_k = 0x7FFFFFFF` and `high_k = 0x80000000`. By observing the code above we see that `range_k` will have value 2 and, by keep iterating, we will end up having `low_k = high_k` which breaks the assumption we made in Elias coding, that is: $high_k > low_k, \forall k$. The breaking of this property will lead to have the decoder to go out of sync and not be able to decode our message.

To address this issue, we start by depicting in the drawing below three different cases of location for the `low_k` and `high_k` quantities.

<img src="converge-cases.png" width="800">

Case **B** denotes a *near convergence* case which might degenerate in having `low_k = high_k`. We note that in such a case we have:
 * Leading bits for `low_k` equal to `01`.
 * Leading bits for `high_k` equal to `10`.
 
We also note that cases **A, C** instead are associated with *normal convergence* whereas our encoder will write the MSB being 0 or 1, respectively.

One solution may consist in detecting the near converge converge case and do the following:
 * Remove the **second** MSB from `low_k` and `high_k`.
 * Keep track with a counter that we encountered a near converge case.
 * Shift to the left all bits of `low_k` and `high_k`.
 * Set to 0 and 1 the MSB of `low_k` and `high_k`, respectively.

When the encoder reaches a case of *normal convergence* (i.e. **A, C** in the figure above), it will write the MSB of either `low_k` or `high_k` and then we need to write all the second MSBs accumulated every time we encountered a *near convergence* case. A little thought should convince you that two situations are possible:
 * Converge case **A**, we write a '0' (MSB of `high_k`) and during all *near converge* cases its MSB was '1'.
 * Converge case **C**, we write a '1' (MSB of `low_k`) and during all *near converge* cases its MSB was '0'.
 
Accordingly, when we write the MSB in **A, C**, we also need to write as many as bits as the number of times we bumped into a *near convergence* case. The value for these accumulated bits is simply the flipped value of the MSB being written.

The Python cell below shows a working solution of an arithmetic encoder.

In [ ]:
import numpy as np
pmf = {'0': [0, 25], '1': [25, 100]}
precision = np.uint32(100)
message = '0110'
low_k, high_k = np.uint32(0), np.uint32(0xFFFFFFFF)
output_buffer = ''
pending_bits = 0

def append_bit_and_pending(bit):
    global output_buffer
    global pending_bits
    output_buffer += bit
    while pending_bits:
        output_buffer += '0' if bit == '1' else '1'
        pending_bits -= 1

for m in message:
    range_k = high_k - low_k + 1
    high_k = low_k + (range_k * pmf[m][1] // precision)
    low_k = low_k + (range_k * pmf[m][0] // precision)
    while True:
        if high_k < 0x80000000:
            append_bit_and_pending('0')
            low_k <<= 1
            high_k <<= 1
            high_k |= 1
        elif low_k >= 0x80000000:
            append_bit_and_pending('1')
            low_k <<= 1
            high_k <<= 1
            high_k |= 1
        elif 0x40000000 <= low_k and high_k < 0xC0000000:
            pending_bits += 1
            low_k <<= 1
            low_k &= 0x7FFFFFFF
            high_k <<= 1
            high_k |= 0x80000001
        else:
            break
            
# Write out the remaining bits
if pending_bits:
    if low_k < 0x40000000:
        append_bit_and_pending('0')
    else:
        append_bit_and_pending('1')

print(f"Output buffer: {output_buffer}")

For completeness we also report the decoder which, without loss of generality, assumes floating point precision when computing the point where our codeword lies in the probability range.

In [ ]:
def get_symbol(current_interval_point, pmf, precision):
    for s in pmf.keys():
        if current_interval_point <= pmf[s][1] / precision:
            return s
    raise Exception("Wrong current_interval_point")

codeword = '0010'
pmf = {'0': [0, 25], '1': [25, 100]}
precision = 100
high_k = 0xFFFFFFFF
low_k = 0
value = 0
symbols_decoded = 0
message = ''
for i in range(32):
  value <<= 1
  if codeword:
    current_bit = codeword[0]
    value += 1 if current_bit == '1' else 0
    codeword = codeword[1:]

while symbols_decoded < 4:
  range_k = high_k - low_k + 1
  count =  ((value - low_k) ) / range_k
  symbol = get_symbol(count, pmf, precision)
  message += symbol
  symbols_decoded += 1
  high_k = low_k + (range_k * pmf[symbol][1]) // precision
  low_k = low_k + (range_k * pmf[symbol][0]) // precision
  while True:
    if ( low_k >= 0x80000000 or high_k < 0x80000000 ):
      low_k <<= 1
      high_k <<= 1
      high_k |= 1
      value <<= 1
      if codeword:
        current_bit = codeword[0]
        value += 1 if current_bit == '1' else 0
        codeword = codeword[1:]
    elif low_k >= 0x40000000 and high_k < 0xC0000000:
      low_k <<= 1
      low_k &= 0x7FFFFFFF
      high_k <<= 1
      high_k |= 0x80000001
      value <<= 1
      if codeword:
        current_bit = codeword[0]
        value += 1 if current_bit == '1' else 0
        codeword = codeword[1:]
    else:
      break

print(f"Decoded message: {message}")

## Getting even more real: Multiplier free implementation
We managed to build a practical implementation of an arithmetic codec which uses integer arithmetic. However, this implementation still uses two multiplications to compute the value of `low_k` and `high_k`. We could save one multiplication if we would use `range_k` in our calculations as depicted in the figure below.

<img src="range-refactor.png" width="800">

However, we still have one multiplication which might be problematic in some hardware (and software) implementations. We can address the removal of this multiplication in two ways:
 * Quantise the values of `range_k` resulting from the multiplication with $p_0$ or $(1 - p_0)$ using a LUT (the CABAC in H.264/AVC, H.265/HEVC and H.266/VVC uses this approach).
 * Approximate the multiplication assuming that the value of range is always close to 1 (the QM encoder in JPEG, MQ encoder in JPEG2000 and arithmetic codec in the MPEG EVC standards use this approach).
 
We'll explain the second approach since in the next notebook we'll use the associated coding engine to perform arithmetic coding over transform coefficients.

Accordingly, if we assume that the value for `range_k` is close to 1 (or `0xFFFFFFFF` in terms of integer arithmetic), we'd have:
 * When a 0 is read from the input, then `range_k = p0`.
 * When a 1 is read from the input, then `range_k = range_k - p0`.

These new approximations do not require any multiplication, so complexity would be saved. Needless to say that the price to pay for this approximation is a loss in coding efficiency.

Aside from the changes required in our core algorithm to make the update of range according to the previous formulae, we also need to make sure that the value of `range_k` stays close to one (remember that `range_k` keeps shrinking are encoding progresses).

To keep the value of range close to 1, *renormalisation* is applied by doubling the value of `range_k`. Doubling is used so in hardware it can be implemented with a simple left shift.

So when should renormalisation take place? A good compromise value is represented by 0.75. In fact, anytime `range_k` goes below that value, multiplying it by two brings it close to 1.

Sadly, we're not done yet. In fact, the update equation below might lead to a very serious problem:
 * When a 1 is read from the input, then `range_k = range_k - p0`
 
`range_k` might then become negative and even if we then apply renormalisation, a left shift over a negative number has an undefined behaviour. This situation could have been prevented if `range_k = p0` but this would imply that we need to read a 0 from the input.

Things would be sorted if we could swap the *semantics* associated with 0 and 1. In fact, rather than thinking in terms of 0s and 1s, we can think of Least Probable Symbol (LPS) and Most Probable Symbol (MPS).

Reasoning in terms of LPS and MPS doesn't change the coding efficiency and more importantly the implementation of our arithmetic codec. Indeed we are still dealing with a binary alphabet. The main difference is that the encoder is now processing the *binary* decision: *is the current symbol the MPS?*, whereby the MPS can be 0 or 1. 

Given that we deal with LPS and MPS, we will not have `p0` and `p1` but rather $p_{LPS}$ and $p_{MPS} = 1 - p_{LPS}$. The figure below shows how the LPS and MPS can be rearranged.

<img src="lps-mps-rearrangement.png" width="600">

We note that the condition which might turn the range to become negative can be detected when `range_k` - $p_{LPS} < p_{LPS}$, that is `range_k / 2` $< p_{LPS}$. This would imply that $p_{MPS} < p_{LPS}$ which is not possible because of the definition of MPS.

In such a case, we can switch the LPS and MPS so that the value for our `range_k` will never become negative. This switch is called *conditional exchange* and, to allow for correct decoding, its condition should also be known at the decoder side.

## Conclusive remarks
We now got in place an entropy coding engine which performs arithmetic coding (a.k.a. Elias coding) using integer arithmetic. Moreover, we've also shown a practical way to build a multiplier free variant, which is the one described in Annex D of JPEG standard [ITU-T T.81](https://www.itu.int/rec/T-REC-T.81) (collectively known as the **JPEG standard**). If our aim were to just describe the implementation of a practical arithmetic coding engine, we would be done now. However, the achievement of the Shannon's entropy by an arithmetic encoder can only be guaranteed if we are able to model correctly the probability of our binary source (or in general our source's *PMF*). Accordingly, here it comes the engineering bit of the arithmetic encoding process which, not surprisingly, can be regarded as a sort of *art and craft*. We'll delve into this subject in the following Jupyter notebook belonging to this tutorial on arithmetic coding.

Another remark is on the integer precision implementation of arithmetic coding with multiplication. This essentially resembles to the well-known [range encoder](https://en.wikipedia.org/wiki/Range_encoding) investigated by N. Martin in 1979. Such a range encoder has then been simplified by the so-called Asymmetric Numerical System (ANS)  entropy coding, which is a clever way of looking at arithmetic coding from a different perspective. A flavour of ANS is currently under consideration in JPEG for the activities related to the **[JPEG-XL](https://www.spiedigitallibrary.org/proceedings/Download?fullDOI=10.1117%2F12.2529237)** standard.